In [ ]:
# Phase 1: Setup and Configuration
# Cell 1: Core Imports and Response Structure
import os
import time
import json
import requests
import logging
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure for therapeutic context"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

# Cell 2: Ollama Client Implementation
class OllamaClient:
    """Robust Ollama client with configurable timeouts"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        """Enhanced JSON parsing with fallback"""
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
            
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Model verification with status checks"""
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        """Model pulling with progress tracking"""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Generation with configurable timeout and retries"""
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# Cell 3: Base Agent Framework
class BaseAgent:
    """Timeout-aware base agent"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300
        
    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        """Generation with time budget tracking"""
        start_time = time.time()
        error_state = False
        timeout_occurred = False
        
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
            
        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)
                    
                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
                
        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )



TherapeuticResponse(text="The old man sat on his porch, rocking in the chair as he watched the world go by. The sun shone brightly overhead, illuminating the grass and trees below. He sighed deeply, remembering happier times when life was simpler and easier to understand.\n\nLife had not always been kind to him. When he was younger, growing up in a small town, everyone knew each other's business. Rumors spread like wildfire and gossip followed closely behind. The old man learned early on that it was better not to get too close to anyone or become too attached. It only led to pain in the end.\n\nNow, as an older man with more life experience under his belt, he understood that humans were complex creatures and had their own demons to battle. Some people struggled with addiction, others dealt with loss and grief, while still others faced loneliness and isolation. He knew firsthand how difficult it could be to navigate these challenges alone.\n\nThe old man's mind wandered back to a time w

In [ ]:
# Phase 1: Setup and Configuration

# Cell 1: Core Imports and Response Structure
import os
import time
import json
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure for therapeutic context"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

# Cell 2: Ollama Client Implementation
class OllamaClient:
    """Robust Ollama client with configurable timeouts"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        """Enhanced JSON parsing with fallback"""
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
            
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Model verification with status checks"""
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        """Model pulling with progress tracking"""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Generation with configurable timeout and retries"""
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# Cell 3: Base Agent Framework
class BaseAgent:
    """Timeout-aware base agent"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300
        
    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        """Generation with time budget tracking"""
        start_time = time.time()
        timeout_occurred = False
        
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
            
        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)
                    
                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
                
        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )

# Cell 4: Prompt Integration and Saving the Response to a Pickle File

# Define the prompt
prompt = "How can I practice mindfulness in daily life?"

# Initialize the Ollama client and the base agent
client = OllamaClient()
agent = BaseAgent(client)

# Generate the therapeutic response using the prompt
response = agent.safe_generate(prompt)

# Save the response object to a pickle file
with open("therapeutic_response.pkl", "wb") as f:
    pickle.dump(response, f)

print("Response saved to therapeutic_response.pkl")


Response saved to therapeutic_response.pkl


In [ ]:
# Phase 2: Core System Components - Sex Positive Focus
# Cell 4: User Prompt and Immediate Response
def sample_sex_positive_query():
    """Example of sex-positive context analysis"""
    client = OllamaClient()
    analyzer = PatientContextAnalyzer(client)
    
    user_query = "We want to explore new ways to connect physically but feel stuck"
    context, _ = analyzer.analyze_context(user_query, [])
    
    print("\nSample Sex-Positive Analysis:")
    print(f"Key Themes: {context.get('key_themes', [])[:3]}")
    print(f"Suggested Approaches: {context.get('suggested_approaches', [])[:2]}")

# Execute sample analysis immediately
sample_sex_positive_query()

# Cell 5: Intimacy Context Analyzer
class PatientContextAnalyzer(BaseAgent):
    """Focuses on sexual dynamics and connection opportunities"""
    def analyze_context(self, input_text: str, history: List[str]) -> Tuple[Dict, float]:
        prompt = f"""Analyze intimacy context (sex-positive focus):
        Couple's Statement: "{input_text[:2000]}"
        History: {" | ".join(history[-3:])[:1000]}
        
        Identify:
        - Communication style about intimacy
        - Expressed/unexpressed desires
        - Emotional blocks
        - Opportunities for exploration
        - Sex-positive reinforcement opportunities
        
        Output JSON with:
        - communication_style: str
        - desires: List[str]
        - emotional_blocks: List[str]
        - exploration_opportunities: List[str]
        - affirmation_strategies: List[str]"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text), response.processing_time

# Cell 6: Intimacy Response Generator
class TherapeuticResponseGenerator(BaseAgent):
    """Generates sex-positive, exploration-focused responses"""
    def generate_response(self, context: Dict, history: List[str]) -> TherapeuticResponse:
        prompt = f"""Create intimacy-enhancing response:
        Context: {json.dumps(context)[:2000]}
        
        Guidelines:
        - Celebrate sexual diversity
        - Normalize exploration desires
        - Suggest creative non-penetrative options
        - Emphasize mutual consent
        - Recommend communication exercises
        - Use affirmative language
        
        Example: "It's wonderful you're wanting to deepen your connection. Many couples find..."
        
        Response:"""
        
        response = self.safe_generate(prompt)
        return self._enhance_response(response)

    def _enhance_response(self, raw_response: TherapeuticResponse) -> TherapeuticResponse:
        """Add intimacy-focused metrics"""
        affirmation_score = min(1.0, raw_response.text.count("healthy")*0.05 + 
                              raw_response.text.count("normal")*0.07)
        return TherapeuticResponse(
            **vars(raw_response),
            empathy_score=affirmation_score,
            refinement_suggestions=[]
        )

# Cell 7: Intimacy Alignment Checker
class ClinicalSafetyChecker(BaseAgent):
    """Ensures responses align with modern sex-positive practices"""
    def evaluate_response(self, response: str, context: Dict) -> Dict:
        prompt = f"""Evaluate intimacy response:
        Response: "{response[:2000]}"
        
        Check for:
        - Judgment language
        - Prescriptive suggestions
        - Assumption of norms
        - Consent emphasis
        - Affirmation quality
        - Inclusive language
        
        Output JSON with:
        - alignment_score: float (0-1)
        - strengths: List[str]
        - improvement_areas: List[str]
        - suggested_alternatives: List[str]"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)

# Cell 8: Integrated Intimacy System
class TherapeuticResponseSystem:
    """Sex-positive couples therapy system"""
    def __init__(self):
        self.client = OllamaClient(model_name="llama2:13b")
        self.agents = {
            'context': PatientContextAnalyzer(self.client),
            'generator': TherapeuticResponseGenerator(self.client),
            'alignment': ClinicalSafetyChecker(self.client)
        }
        self.conversation_history = []
        
    def process_session(self, input_text: str) -> Dict:
        """Focus on intimacy enhancement pipeline"""
        result = {
            'response': '',
            'context_analysis': {},
            'alignment_check': {},
            'timings': {}
        }
        
        try:
            # Context analysis
            ctx_start = time.time()
            context, _ = self.agents['context'].analyze_context(input_text, self.conversation_history)
            result['context_analysis'] = context
            
            # Generate response
            gen_start = time.time()
            response = self.agents['generator'].generate_response(context, self.conversation_history)
            result['timings']['generation'] = time.time() - gen_start
            result['response'] = response.text
            
            # Alignment check
            alignment_start = time.time()
            alignment_check = self.agents['alignment'].evaluate_response(response.text, context)
            result['alignment_check'] = alignment_check
            result['timings']['alignment'] = time.time() - alignment_start
            
            # Update history
            self._update_history(input_text, result['response'])
            
        except Exception as e:
            result['error'] = str(e)
        
        return result

    def _update_history(self, input_text: str, response: str):
        """Maintain intimacy-focused history"""
        self.conversation_history.extend([
            f"Couple: {input_text[:300]}",
            f"Therapist: {response[:300]}"
        ])[-6:]

In [ ]:
# Phase 3: Testing - Cell 9
def test_intimacy_scenario():
    system = TherapeuticResponseSystem()
    result = system.process_session("We're bored with our sex life but nervous about trying new things")
    
    print("\nIntimacy Enhancement Response:")
    print(result['response'][:500])
    print("\nSuggested Strategies:")
    print(result['context_analysis'].get('affirmation_strategies', [])[:2])

if __name__ == "__main__":
    test_intimacy_scenario()

In [ ]:
# Phase 3: Testing and Validation
# Cell 8: Test Scenarios
def test_empty_input():
    """Test handling of empty patient input"""
    system = TherapeuticResponseSystem()
    result = system.process_session("")
    print("\nTest 1 - Empty Input:")
    print(f"Response: {result['response'][:150]}")
    print(f"Errors: {result['errors']}")
    print(f"Crisis Alert: {result['crisis_alert']}")

def test_crisis_detection():
    """Test suicide risk detection"""
    system = TherapeuticResponseSystem()
    result = system.process_session("I can't take it anymore, I have a bottle of pills ready")
    print("\nTest 2 - Crisis Detection:")
    print(f"Response: {result['response'][:250]}")
    print(f"Crisis Alert: {result['crisis_alert']}")
    print(f"Safety Concerns: {result['safety_check'].get('concerns', [])[:2]}")

def test_invalid_input_type():
    """Test non-string input handling"""
    system = TherapeuticResponseSystem()
    result = system.process_session(12345)
    print("\nTest 3 - Invalid Input Type:")
    print(f"Response: {result['response'][:150]}")
    print(f"Errors: {res
    ult['errors']}")

# Cell 9: Test Execution
if __name__ == "__main__":
    test_empty_input()
    test_crisis_detection()
    test_invalid_input_type()

2025-02-20 02:05:37,440 - WARNING - Generation timed out (attempt 1)
2025-02-20 02:05:37,782 - ERROR - Generation timed out after 300s
2025-02-20 02:10:38,031 - WARNING - Generation timed out (attempt 1)
2025-02-20 02:10:38,171 - ERROR - Generation timed out after 300s
